In [1]:
import numpy as np
import h5py
import pickle
from DDQN_model_new import Environment, Agent

Using TensorFlow backend.


In [10]:
def DDQN_Simulation(agent, env, test_model, size , loads):
    total_reward = 0
    if size == 3:
        current_state = (0,1,1,[0,0,0,0,0,0,0,0,0])
    else:
        current_state = (0,1,1,[0,0,0,0])
        
    trailer_loc, day_of_week, trailer_size, load_vector = current_state # initial state

    for i in range(1000):

    #     print('initial state:',current_state)

        state_flatten = [] # flatten states
        for j in range(3):
            state_flatten.append(current_state[j])
        for k in range((size*size)):
            state_flatten.append(current_state[3][k])

        act_values = test_model.predict(np.array(state_flatten).reshape(1, agent.state_space))
        predicted_optimal_action = np.argmax(act_values[0]) # action that will produce the best value
    #     print("action is", predicted_optimal_action)

        current_state_encode = env.get_numeric_state_nomadic_trucker(trailer_loc, day_of_week, trailer_size, load_vector)
        reward = env.get_rewards(current_state_encode, predicted_optimal_action) # get reward from action
    #     print("reward", reward)
        total_reward += reward


        next_truck_location, next_day, next_trailer_type, next_load_vector = env.get_actual_next_grid_state(predicted_optimal_action, current_state_encode, is_test) # go to the next state
        next_load_vector = loads[i]

        if next_load_vector[next_truck_location] == 1:
            next_load_vector[next_truck_location] = 0


        next_state = next_truck_location, next_day, next_trailer_type, next_load_vector
        current_state = next_state  # get new state
        trailer_loc, day_of_week, trailer_size, load_vector = current_state

    return(total_reward)


In [16]:
# possible states
s = [3]
# s = [2]


# possible days
d = [1, 3, 7]

# possible trailer types
t = [1, 3, 3]

#Batch_size for 2x2
# Batch = [15, 32, 64]

#Batch_size for 3x3
Batch = [32, 64, 128]


In [15]:
for size in s:
    for days, trailers in zip(d,t):
        for batch_size in Batch:
            num_actions = size*size
            state_inputs = 3 + num_actions
            state_space_size = (size*size)*days*trailers*(2**(size*size))
            my_agent = Agent(num_actions, state_inputs, batch_size, state_space_size)  # number of actions, number of state inputs (when state is flattened)
            my_env = Environment(my_agent, size, days, trailers)
            my_test_model = my_agent.build_test_model(state_space_size, batch_size)
            is_test = True

            loads = pickle.load(open("Load_"+str(size)+"_"+str(days)+".pickle", "rb"))        
            Result = DDQN_Simulation(my_agent, my_env, my_test_model, size, loads)
            print("Result for",str(state_space_size), "states, batch size",batch_size,":", Result)
            

Result for 4608 states, batch size 32 : 145756.83520000006
Result for 4608 states, batch size 64 : 192799.25245666734
Result for 4608 states, batch size 128 : 141288.30304333332
Result for 41472 states, batch size 32 : 310187.02425333345
Result for 41472 states, batch size 64 : 279915.9701766666
Result for 41472 states, batch size 128 : 137974.5368433332
Result for 96768 states, batch size 32 : 325291.88068666693
Result for 96768 states, batch size 64 : 265722.6751399999
Result for 96768 states, batch size 128 : 111703.52587666658
